In [1]:
import sys
sys.setrecursionlimit(10000)
from collections import deque
import heapq

### Model (search problem)

class TransportationProblem(object):
    def __init__(self, N):
        # N = number of blocks
        self.N = N

    def startState(self):
        return 1

    def isEnd(self, state):
        return state == self.N

    def succAndCost(self, state):
        # return list of (action, newState, cost) triples
        result = []

        # Walk to the next block
        if state + 1 <= self.N:
            result.append(('walk', state + 1, 1))

        # Run to the block 3 steps ahead
        if state + 3 <= self.N:
            result.append(('run', state + 3, 1.5))

        # Take the magic tram to 2 * current block
        if state * 2 <= self.N:
            result.append(('tram', state * 2, 2))

        return result

### Algorithms

def printSolution(solution):
    totalCost, history = solution
    print('totalCost: {}'.format(totalCost))
    for item in history:
        print(item)



# Dynamic Programmming


In [2]:
def dynamicProgramming(problem):
    cache = {} # state -> futureCost(state), action, newState, cost
    def futureCost(state):
        # Base case
        if problem.isEnd(state):
            return 0
        if state in cache: # Exponential savings
            return cache[state][0]
        # Actually doing work
        result = min((cost+futureCost(newState), action, newState, cost) \
                for action, newState, cost in problem.succAndCost(state))
        cache[state] = result
        return result[0]

    state = problem.startState()
    totalCost = futureCost(state)

    # Recover history
    history = []
    while not problem.isEnd(state):
        _, action, newState, cost = cache[state]
        history.append((action, newState, cost))
        state = newState

    return (futureCost(problem.startState()), history)

# Breath First Search
We Know that in breath first search we assume the cost to be constant c and constant is greater than 0 (c>0). We traverse the tree Level wise

In [3]:
def bfs(problem):
    queue = deque([(problem.startState(), [])])

    while queue:
        state, history = queue.popleft()

        if problem.isEnd(state):
            return (sum(cost for _, _, cost in history), history)

        for action, newState, cost in problem.succAndCost(state):
            queue.append((newState, history + [(action, newState, cost)]))
            
    # return (float('+inf'), None)


# Depth First Search
As we know that is depth first search we assume the cost to be 0 at each edge, and we traverse the whole tree once

In [4]:
def dfs(problem):
    def recurse(state, history, totalCost):
        if problem.isEnd(state):
            return totalCost, history

        for action, newState, cost in problem.succAndCost(state):
            result = recurse(newState, history + [(action, newState, cost)], totalCost + cost)
            if result:
                return result

    return recurse(problem.startState(), history=[], totalCost=0)


# Uniform Cost Search


In [5]:
import heapq
class PriorityQueue:
    def __init__(self):
        self.DONE = -100000
        self.heap = []
        self.priorities = {} #Map from state to priority
    #Insert [state] into the heap with priority [newPriority] if [state] insn't
    #in the heap or [newPriority] is smaller than the exisiting priority
    def update(self, state , newPriority):
        oldPriority = self.priorities.get(state)
        if oldPriority == None or newPriority < oldPriority:
            self.priorities[state] = newPriority
            heapq.heappush(self.heap, (newPriority ,state))
            return True
        return False

    def removeMin(self):
        while len(self.heap) > 0:
            priority ,state = heapq.heappop(self.heap)
            if self.priorities[state] == self.DONE : continue #Outdated Priority skip
            self.priorities[state] = self.DONE
            return(state, priority)
        return (None, None) #Nothing Left


def uniformCostSearch(problem):
    frontier = PriorityQueue()
    frontier.update(problem.startState(), 0)

    # Maintain history alongside cost
    history = {problem.startState(): (0, [])}

    while True:
        # Move frontier to Explored
        state, pastCost = frontier.removeMin()
        if problem.isEnd(state):
            return history[state]

        # Push out of Frontier
        for action, newState, cost in problem.succAndCost(state):
            newCost = pastCost + cost
            if frontier.update(newState, newCost):
                # Update history only if the state is new or has a lower cost
                history[newState] = (newCost, history[state][1] + [(action, newState, cost)])


In [6]:
### Main

problem = TransportationProblem(N=10)  

print("\nDynamic Programming:")
printSolution(dynamicProgramming(problem))

print("\nBreadth-First Search:")
printSolution(bfs(problem))

print("\nDepth-First Search:")
printSolution(dfs(problem))

print("\nUniform Cost Search:")
printSolution(uniformCostSearch(problem))



Dynamic Programming:
totalCost: 4.5
('run', 4, 1.5)
('run', 7, 1.5)
('run', 10, 1.5)

Breadth-First Search:
totalCost: 4.5
('walk', 2, 1)
('run', 5, 1.5)
('tram', 10, 2)

Depth-First Search:
totalCost: 9
('walk', 2, 1)
('walk', 3, 1)
('walk', 4, 1)
('walk', 5, 1)
('walk', 6, 1)
('walk', 7, 1)
('walk', 8, 1)
('walk', 9, 1)
('walk', 10, 1)

Uniform Cost Search:
totalCost: 4.5
('walk', 2, 1)
('run', 5, 1.5)
('tram', 10, 2)
